<a href="https://colab.research.google.com/github/AnastasiaZhivilo/Data-Science-Projects/blob/main/Crop_Boundary_Recognition/GeoTIFF_export_for_training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialise Google Drive and the Google Project cs88-2-capstone

## Please enable GPU runtime in Colab

In [ ]:
#pip install rasterio

In [ ]:
# Import necessary libraries
import ee
import geemap
import ee.batch
import os
from google.colab import drive
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Import the necessary component directly from ipyleaflet
try:
    from ipyleaflet import DrawControl
except ImportError:
    print("ipyleaflet DrawControl could not be imported. Please ensure geemap is installed.")
    # Stop execution if the critical component is missing
    raise


# Authenticate the Google Project and connect to Google Drive

In [ ]:
# 1. Mount Google Drive
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

# 2. Authenticate and initialise Google Earth Engine

ee.Authenticate()
ee.Initialize(project='cs88-2-capstone')
print("Earth Engine authenticated and initialized.")

### Select a central geographical point and define a boundary box

In [ ]:

# 1. Define the central point
center_lon = 146.9561
center_lat = -34.7022
center_point = ee.Geometry.Point(center_lon, center_lat)

# 2. Define the size (radius) for the buffer in meters
buffer_radius_meters = 5000  # 15 kilometers

# 3. Create a geodesic buffer (circle) around the point
circle_geometry = center_point.buffer(buffer_radius_meters)

# 4. Get the Bounding Box (Rectangle) coordinates from the buffered geometry
# .bounds() returns an ee.Geometry.Rectangle
bounding_box_geom = circle_geometry.bounds()

bbox_coords = bounding_box_geom.getInfo()['coordinates'][0]

# 5. Set up a folder name for this bounding box
folder_name = f"BBox_{center_lon:.4f}_{center_lat:.4f}".replace('.', '_').replace('-', '_')

### Select Sentinel-2 data for visualisation purposes, to allow for manual data labeling

In [ ]:

# Set up Cloud masking function:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask).eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


# Define date range
start_date = '2023-09-01'
end_date = '2024-09-15'


# Extract Sentinel-2 data for RGB bands with cloud masking
image = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(start_date, end_date)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4','B3','B2'],
}


In [ ]:
# Define date range
start_date = '2024-07-01'
end_date = '2024-09-15'


### Set up an interactive map in the boundary box that will allow to extract geographic information of the polygons

To use this draw all the necessary polygons on the map, the coordinates will be displayed below the map, these coordinates will be used later to create GeoTIFF files

In [ ]:

# Create a median composite of the filtered image collection
median_image = image.median().clip(bounding_box_geom)
print("Median image composite created and clipped to bounding box.")

# 1. Initialize the geemap interactive map
Map = geemap.Map(center=[center_lat, center_lon], zoom=12)

# 2. Add the clipped median image to the map
Map.add_ee_layer(median_image, visualization, 'Median Sentinel-2 Image')

# 3. Fit the map view to the bounding box
Map.zoom_to_bounds(bounding_box_geom.bounds().getInfo().get('coordinates')[0])
print("Map view adjusted to bounding box.")

# 4. Manually create the DrawControl object with TraitError-compliant arguments 🛠️
# The fix: Pass an empty dictionary {} to disable tools while satisfying the TraitError.
if DrawControl is not None:
    # 🌟 Configuration for the ONLY tool we want enabled: Polygon
    polygon_config = {
        'shapeOptions': {
            'fillColor': '#ff0000',
            'color': '#ff0000',
            'fillOpacity': 0.1,
            'weight': 2
        }
    }

    # Create the DrawControl, using {} to effectively disable tools
    draw_control = DrawControl(
        # The tool we want enabled
        polygon=polygon_config,

        # Tools we must disable by passing an EMPTY DICT {}
        polyline={}, # Disables polyline
        rectangle={}, # Disables rectangle
        circle={}, # Disables circle
        marker={}, # Disables marker

        # Control options
        edit=False,
        remove=False,
        position='topleft'
    )

    # Add the manually created control to the geemap object
    Map.add_control(draw_control)
    print("Drawing tools added to the map (Polygon only).")

    # 5. Define a function to execute when a drawing is completed
    def handle_draw(target, action, geo_json):
        """
        Callback function that runs when a user finishes drawing a feature.
        """
        # The geo_json argument contains the GeoJSON of the drawn feature
        if geo_json:
            geometry = geo_json.get('geometry')
            coordinates = geometry.get('coordinates')
            geometry_type = geometry.get('type')

            print(f"\n--- Drawn {geometry_type} Coordinates ---")
            if geometry_type == 'Polygon':
                # For a Polygon, the exterior ring coordinates are the first element of the list
                print(f"Exterior Ring Coordinates (GeoJSON format): {coordinates[0]}")
            else:
                print(f"Coordinates (GeoJSON format): {coordinates}")
            print("----------------------------------------")

            # Since the drawn feature is stored in geo_json, we don't need to manually clear a buffer.

    # 6. Attach the callback function to the control's 'on_draw' event
    draw_control.on_draw(handle_draw)

# 7. Display the map
print("Displaying interactive map. Use the Polygon tool (top-left) to draw polygons.")
Map

### Create a function that will convert polygon coordinates into a GeoTIFF file

In [ ]:
import ee

resolution_meters = 10                   # Spatial resolution of the GeoTIFF (e.g., 10m for Sentinel-2 data)

# --- GEOFFI EXPORT LOGIC (UPDATED FOR INSTANCE SEGMENTATION) ---

def export_polygon_to_geotiff(coords, name, folder, scale):
    """
    UPDATED: Exports the ENTIRE FILLED POLYGON AREA (not just the boundary line)
    as a GeoTIFF. This is essential for generating the instance segmentation
    Distance Map in the training script.
    """
    try:
        # 1. Create the Earth Engine Polygon Geometry
        ee_polygon = ee.Geometry.Polygon(coords)
        print(f"\n--- Starting Export for {name} ---")

        # CONVERT GEOMETRY TO FEATURE COLLECTION:
        ee_feature_collection = ee.FeatureCollection([ee.Feature(ee_polygon)])

        # 2. Rasterize the Geometry (Paint it onto an Image)
        empty_image = ee.Image.constant(0).rename('field_interior')

        # Paint the feature collection onto the image, assigning a value (e.g., 1)
        # The 'width' parameter is removed, causing the polygon to be FILLED.
        rasterized_image = empty_image.paint(
            featureCollection=ee_feature_collection,
            color=1
            # width=1 is removed here
        ).reproject(crs='EPSG:4326', scale=scale)

        print("Filled geometry rasterized successfully.")

        # 3. Define Export Parameters and Start Task
        # Note: The output is now the 'filled' area for a single field instance.
        export_task = ee.batch.Export.image.toDrive(
            image=rasterized_image,
            description=name,
            folder=folder,
            fileNamePrefix=name,
            region=ee_polygon.bounds(),
            scale=scale,
            fileFormat='GeoTIFF'
        )

        export_task.start()

        print(f"Task Name: {name} STARTED (Exporting Filled Area).")

    except Exception as e:
        print(f"\nAn error occurred during export of {name}: {e}")


### Create a function that will enable batch processing of multiple polygons in separate GeoTIFF files

In [ ]:

# Iterate over the list of polygons and start an export task for each one.
def batch_processing_polygons(polygons_to_export):
    print("--- Starting Batch GeoTIFF Export Tasks ---")
    for polygon in polygons_to_export:
        export_polygon_to_geotiff(
            coords=polygon['coordinates'],
            name=polygon['name'],
            folder=folder_name,
            scale=resolution_meters
        )
    print("\nAll export tasks have been initiated.")
    print("Check the Colab 'Tasks' tab or run 'ee.batch.data.getTaskList().getInfo()' to monitor progress.")




###

### Define the polygons in GeoJSON with Gemini AI's help to speed up the process

In [ ]:
# This part is generated by AI. In order to generate paste the Coordinates that appear under the map
# when you create the polygons into a Gemini AI chat with following sentence:

# "Convert the following drawn polygon coordinates into a Python list named polygons_to_export,
#assigning sequential names starting with Polygon_1_Export:""

# For the coordinates copy them in the same format that appears under the map. ie:

# --- Drawn Polygon Coordinates ---
# Exterior Ring Coordinates (GeoJSON format): [[146.919866, -34.722905], [146.93377, -34.724915], [146.933341, -34.728724], [146.919136, -34.727067], [146.919866, -34.722905]]
# ----------------------------------------

# --- Drawn Polygon Coordinates ---
# Exterior Ring Coordinates (GeoJSON format): [[146.918192, -34.732534], [146.926904, -34.733592], [146.927848, -34.728231], [146.919136, -34.727137], [146.918192, -34.732534]]
# ----------------------------------------

#Then paste the output here.

# polygons_to_export = [
#     {
#         'name': 'Polygon_A_Export',
#         'coords': [
#             [146.889825, -34.768324], [146.896262, -34.769452], [146.897206, -34.763423], [146.891069, -34.762224], [146.889782, -34.767936], [146.889825, -34.768324]
#         ]
#     },
#     {
#         'name': 'Polygon_B_Export',
#         'coords': [
#             [146.89652, -34.769416], [146.903043, -34.770298], [146.903987, -34.76508], [146.897507, -34.763599], [146.89652, -34.769416]
#         ]
#     },
#     {
#         'name': 'Polygon_C_Export',
#         'coords': [
#             [146.903644, -34.770333], [146.909995, -34.771461], [146.910768, -34.766385], [146.904416, -34.764939], [146.903644, -34.770333]
#         ]
#     }
# ]

polygons_to_export = [
    {
        "name": "Polygon_1_Export",
        "coordinates": [
            [146.919866, -34.722905],
            [146.93377, -34.724915],
            [146.933341, -34.728724],
            [146.919136, -34.727067],
            [146.919866, -34.722905]
        ]
    },
    {
        "name": "Polygon_2_Export",
        "coordinates": [
            [146.918192, -34.732534],
            [146.926904, -34.733592],
            [146.927848, -34.728231],
            [146.919136, -34.727137],
            [146.918192, -34.732534]
        ]
    },
    {
        "name": "Polygon_3_Export",
        "coordinates": [
            [146.927161, -34.733627],
            [146.935229, -34.734685],
            [146.936259, -34.729042],
            [146.928191, -34.728266],
            [146.927161, -34.733627]
        ]
    }
]

### Run the batching process. This will create a GeoTIFF file per polygon and save in the folder for the specified bounding box central point.

This may take a few minutes.

In [ ]:
batch_processing_polygons(polygons_to_export)

### To check if the polygons look correct you can view a couple of the GeoTiff files

In [ ]:
# --- View Exported GeoTIFF in Colab ---
# This script requires the following libraries. If they are not installed, run:
# !pip install rasterio matplotlib


# Function to read and plot a single GeoTIFF
def plot_geotiff(file_name, folder_name, drive_path):
    """Reads a GeoTIFF and plots the boundary data."""

    geotiff_path = os.path.join(
        drive_path,
        'MyDrive',
        folder_name,
        f'{file_name}.tif'
    )

    print(f"Looking for file: {file_name}.tif")

    if not os.path.exists(geotiff_path):
        print(f"-> ERROR: File '{file_name}.tif' not found. Ensure the export task is COMPLETED.")
        return

    try:
        with rasterio.open(geotiff_path) as src:
            # Read the boundary band (which contains 1s for the border and 0s elsewhere)
            boundary_data = src.read(1)

            # Count how many non-zero pixels (borders) exist to ensure file isn't empty
            if np.sum(boundary_data) == 0:
                print(f"-> WARNING: File '{file_name}.tif' contains no boundary data (all zeros).")
                return

            # 5. Plot the data
            plt.figure(figsize=(8, 8))

            # Use 'interpolation=None' to show the sharp, 10m pixel boundaries
            # Use a colormap suitable for binary data (like 'Greys')
            plt.imshow(boundary_data, cmap='Greys', interpolation='none')

            plt.title(f'GeoTIFF Visualization: {file_name}.tif')
            plt.xlabel('Pixel Column')
            plt.ylabel('Pixel Row')

            # Since the image is just a border, we only need to show the value 1
            plt.colorbar(label='Boundary Value (1 = Border)', ticks=[0, 1])

            plt.show()
            print(f"Successfully displayed GeoTIFF for {file_name}.")

    except Exception as e:
        print(f"Error processing GeoTIFF file {file_name}: {e}")


In [ ]:

# --- Batch Visualization ---


# The base path where Google Drive is mounted
drive_mount_path = '/content/drive'


# Define the list of file names you want to view.
# UPDATE THIS LIST to match the 'name' field in your polygons_to_export list.
file_names_to_view = [
    'Polygon_1_Export',
    'Polygon_2_Export',
    'Polygon_3_Export'
]

for name in file_names_to_view:
    plot_geotiff(name, folder_name, drive_mount_path)

print("\n--- Batch Visualization Attempt Complete ---")


# Extract other necessary layers for the defined Bounding Box that will be used for training.

Set up date ranges, bands and indicies required from Sentinel-2

In [ ]:

def add_indices(image):
    """
    Calculates NDVI and SAVI, and applies a smoothing filter to NDVI.
    The resulting image will have the new bands: 'NDVI' (smoothed) and 'SAVI'.
    """
    # 1. Calculate NDVI: (B8 - B4) / (B8 + B4)
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI_raw')

    # 2. Smooth NDVI using a 3x3 median filter to reduce noise while preserving edges
    # The kernel is set to a radius of 1 pixel, resulting in a 3x3 window (2*1 + 1)
    ndvi_smoothed = ndvi.focal_median(radius=1, kernelType='square', units='pixels').rename('NDVI')

    # 3. Calculate SAVI: Soil Adjusted Vegetation Index (L=0.5)
    # SAVI = ((B8 - B4) / (B8 + B4 + L)) * (1 + L)
    L = ee.Number(0.5)
    savi = image.expression(
        '((NIR - RED) / (NIR + RED + L)) * (1 + L)', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'L': L
        }).rename('SAVI')

    # Return the original image with the two new derived bands
    return image.addBands(ndvi_smoothed).addBands(savi)


Batch configuration



In [ ]:

# TARGET BANDS now includes the two new indices (Smoothed NDVI and SAVI)
TARGET_BANDS = ['B4', 'B3', 'B2', 'B8', 'NDVI', 'SAVI']
EXPORT_SCALE = 10  # 10 meters resolution

# Define different date ranges for separate GeoTIFF exports
# Adjusted for Southern Hemisphere (NSW Riverina Winter Crops)
DATE_RANGES = [
    # 1a. Canola Peak (Early Spring): Captures the bright yellow flowering stage
    {'start': '2024-09-01', 'end': '2024-09-30', 'name_suffix': 'CanolaPeak'},

    # 1b. Barley Senescence (Mid-Spring): Barley starts ripening, maximizing contrast with green wheat.
    {'start': '2024-10-01', 'end': '2024-10-31', 'name_suffix': 'BarleySenescence'},

    # 1c. Wheat Peak (Late Spring): Wheat is fully developed/filling, while barley is dry.
    {'start': '2024-11-01', 'end': '2024-11-30', 'name_suffix': 'WheatPeak'},

    # 2. Establishment/Early Growth (Winter): Sowing and Emergence stage
    {'start': '2024-05-01', 'end': '2024-07-31', 'name_suffix': 'Emergence'},

    # 3. Post-Harvest/Bare Earth (Summer): Fields are typically bare or harvested
    {'start': '2023-12-01', 'end': '2024-02-29', 'name_suffix': 'BareEarth'}
]

Export function

In [ ]:

def export_sentinel2_composite(date_range_config, geometry, folder_name):
    """
    Filters Sentinel-2 data, creates a median composite with indices, and starts export.
    """
    start_date = date_range_config['start']
    end_date = date_range_config['end']
    name_suffix = date_range_config['name_suffix']

    # 1. Filter and process the ImageCollection
    s2_collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(start_date, end_date)
        .filterBounds(geometry)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
        .map(mask_s2_clouds)
        # Add indices (NDVI, SAVI) and smooth NDVI
        .map(add_indices)
    )

    # 2. Create the median composite and clip to the geometry
    median_composite = s2_collection.median().clip(geometry)

    # 3. Select only the required bands (now 6 total)
    final_image = median_composite.select(TARGET_BANDS)

    # 4. FIX: Cast the entire image to Float32 to ensure compatible data types across all bands
    final_image = final_image.toFloat()

    # 5. Define file name and description
    file_name = f"Sentinel2_{name_suffix}_{start_date.replace('-', '')}_{end_date.replace('-', '')}"

    print(f"\nStarting export for: {name_suffix} ({start_date} to {end_date})")

    # 6. Start the Export Task
    task = ee.batch.Export.image.toDrive(
        image=final_image,
        description=file_name,
        folder=folder_name,       # Dynamic folder name
        fileNamePrefix=file_name,
        region=geometry.bounds(),
        scale=EXPORT_SCALE,
        fileFormat='GeoTIFF',
        formatOptions={'cloudOptimized': True}
    )

    task.start()
    print(f"Task '{file_name}' started. Check Colab Tasks tab for progress.")


Batch Execution

In [ ]:
if __name__ == '__main__':
    # Execute the export for each defined date range
    print("--- Starting Batch Sentinel-2 Export ---")
    for dr in DATE_RANGES:
        export_sentinel2_composite(dr, bounding_box_geom, folder_name)

    print("\nAll export tasks submitted to Earth Engine.")
    print("Monitor the Colab Tasks tab for completion before starting U-Net training. Check progress here: https://console.cloud.google.com/earth-engine/tasks?project=cs88-2-capstone")

In [ ]:
 !pip install scipy first

In [ ]:
# ==============================================================================
# U-NET MODEL FOR CROP BOUNDARY SEGMENTATION (MULTI-HEAD ARCHITECTURE)
# Now adapted to load INDIVIDUAL FILLED POLYGON MASKS (exported from EE)
# and generate the 2-channel Instance Label Map (Boundary + Distance) directly.
# ==============================================================================

import os
import numpy as np
import rasterio
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
import scipy.ndimage as ndi # Required for Distance Transform (Instance Segmentation)
from skimage.transform import resize # Required to align masks to a consistent shape

# --- Configuration ---
# Must match the 5 exports * 6 bands/export = 30 channels
NUM_CHANNELS = 30
TILE_SIZE = 256
BATCH_SIZE = 8
EPOCHS = 25
VALIDATION_SPLIT = 0.1
OUTPUT_CHANNELS = 2 # Boundary Mask (0) and Distance Map (1)

# --- Data Path Configuration ---
# Ensure these match your export parameters
center_lon = 146.9561
center_lat = -34.7022
folder_name = f"BBox_{center_lon:.4f}_{center_lat:.4f}".replace('.', '_').replace('-', '_')
DRIVE_PATH = f'/content/drive/MyDrive/{folder_name}'

# List of all five Sentinel-2 input files to be loaded and stacked
INPUT_IMAGE_NAMES = [
    'Sentinel2_CanolaPeak_20240901_20240930',
    'Sentinel2_BarleySenescence_20241001_20241031',
    'Sentinel2_WheatPeak_20241101_20241130',
    'Sentinel2_Emergence_20240501_20240731',
    'Sentinel2_BareEarth_20231201_20240229'
]

# The GeoTIFF mask files (labels) exported from the GeoJSON (must be FILLED polygons now)
MASK_FILE_NAMES = [
    'Polygon_1_Export',
    'Polygon_2_Export',
    'Polygon_3_Export'
    # Add all other polygon mask exports here
]

In [ ]:

# ==============================================================================
# 2. DATA UTILITY FUNCTIONS
# ==============================================================================

def load_all_input_tiffs(image_names, path):
    """Loads all five Sentinel-2 time-series GeoTIFFs and stacks them."""
    stacked_images = []

    # Load all 5 time-step images and stack them along the channel axis
    for name in image_names:
        file_path = os.path.join(path, f'{name}.tif')
        if not os.path.exists(file_path):
            print(f"Error: Input file not found: {file_path}")
            return None

        with rasterio.open(file_path) as src:
            # Read all 6 bands for this time step: (6, H, W)
            img = src.read()
            # Transpose to (H, W, 6) and append to stack
            stacked_images.append(img.transpose((1, 2, 0)))

    # Concatenate all 5 time steps: (H, W, 5 * 6) = (H, W, 30)
    full_img_stack = np.concatenate(stacked_images, axis=2)
    print(f"Successfully loaded and stacked input image with shape: {full_img_stack.shape}")

    return full_img_stack


def generate_instance_labels(mask_names, path, target_shape):
    """
    Loads individual FILLED polygon masks, aligns them, generates their distance maps,
    and combines everything into the final 2-channel label array (Boundary + Distance).
    """
    target_h, target_w = target_shape[0], target_shape[1]

    # Initialize the final label arrays
    final_boundary_mask = np.zeros(target_shape, dtype=np.float32)
    final_distance_map = np.zeros(target_shape, dtype=np.float32)

    for name in mask_names:
        file_path = os.path.join(path, f'{name}.tif')
        if not os.path.exists(file_path):
            print(f"Warning: Mask file not found: {file_path}. Skipping.")
            continue

        with rasterio.open(file_path) as src:
            # Read the single-band mask (Filled Polygon): (1, H, W)
            mask_filled = src.read(1)

            # 1. ALIGN MASK TO TARGET SHAPE (H, W)
            if mask_filled.shape != target_shape:
                mask_filled = resize(
                    mask_filled,
                    target_shape,
                    order=0, # Nearest-neighbor interpolation
                    preserve_range=True,
                    anti_aliasing=False
                ).astype(np.uint8)

            # Normalize the mask to be 1.0 (inside) or 0.0 (outside)
            mask_filled = (mask_filled > 0).astype(np.float32)

            # 2. GENERATE BOUNDARY MASK (Channel 0)
            # Find the boundary by subtracting the eroded mask from the original filled mask.
            # Erode the mask by one pixel (or use a different method if preferred)
            eroded_mask = ndi.binary_erosion(mask_filled, iterations=1)
            field_boundary = mask_filled - eroded_mask

            # Combine this field's boundary with the final boundary mask (logical OR)
            final_boundary_mask = np.maximum(final_boundary_mask, field_boundary)

            # 3. GENERATE DISTANCE MAP (Channel 1)
            # Compute Euclidean Distance Transform from the interior (value 1) to the boundary (value 0).
            distance_to_boundary = ndi.distance_transform_edt(mask_filled)

            # Combine distance maps: We want the *maximum* distance for the center of the fields.
            final_distance_map = np.maximum(final_distance_map, distance_to_boundary)


    # 4. Final Normalization and Stacking

    # Normalize the combined distance map to 0-1 range
    distance_max = final_distance_map.max()
    if distance_max > 0:
        final_distance_map /= distance_max

    # Stack the two channels: (H, W, 2)
    label_2ch = np.dstack([
        np.expand_dims(final_boundary_mask, axis=-1),
        np.expand_dims(final_distance_map, axis=-1)
    ]).astype(np.float32)

    print(f"Successfully generated 2-channel label map with shape: {label_2ch.shape}")
    return label_2ch


def create_tiles(full_img, full_mask_2ch, tile_size):
    """Slides a window across the full image/mask to create fixed-size training tiles."""
    height, width, _ = full_img.shape
    X_tiles = []
    Y_tiles = []

    # Iterate with a non-overlapping sliding window
    for y in range(0, height - tile_size + 1, tile_size):
        for x in range(0, width - tile_size + 1, tile_size):
            # Extract image tile (X)
            img_tile = full_img[y:y + tile_size, x:x + tile_size, :]

            # Extract the 2-channel mask tile (Y)
            mask_tile = full_mask_2ch[y:y + tile_size, x:x + tile_size, :]

            # Only include tiles that contain at least one boundary pixel (for efficiency)
            if np.any(mask_tile[:, :, 0]):
                X_tiles.append(img_tile)
                Y_tiles.append(mask_tile)

    X_train = np.array(X_tiles)
    Y_train = np.array(Y_tiles)

    print(f"Tiling complete. Generated {len(X_tiles)} training tiles.")
    return X_train, Y_train


In [ ]:

# Custom Loss Function to handle the two different output types
def multi_head_loss(y_true, y_pred):
    """
    Loss function for the 2-channel output:
    Channel 0 (Boundary Mask): Binary Crossentropy
    Channel 1 (Distance Map): Mean Squared Error
    """
    # Channel 0: Boundary Mask (Binary Segmentation)
    y_true_boundary = y_true[:, :, :, 0]
    y_pred_boundary = y_pred[:, :, :, 0]

    # Calculate Binary Crossentropy. Output shape: (batch, H, W)
    boundary_loss_per_pixel = K.binary_crossentropy(y_true_boundary, y_pred_boundary)
    # CRITICAL FIX: Take the mean over the spatial dimensions (H and W) to get shape (batch,)
    boundary_loss = K.mean(boundary_loss_per_pixel, axis=[1, 2])

    # Channel 1: Distance Map (Regression)
    y_true_distance = y_true[:, :, :, 1]
    y_pred_distance = y_pred[:, :, :, 1]

    # Mean Squared Error for the distance prediction. Output shape: (batch, H, W)
    distance_loss_per_pixel = K.square(y_pred_distance - y_true_distance)
    # CRITICAL FIX: Take the mean over the spatial dimensions (H and W) to get shape (batch,)
    distance_loss = K.mean(distance_loss_per_pixel, axis=[1, 2])

    # Combine the losses (now both are the compatible shape (batch,))
    total_loss = boundary_loss + (0.5 * distance_loss)
    return total_loss

def build_unet(input_shape):
    """Defines a Multi-Head U-Net architecture for instance segmentation hint prediction."""
    inputs = Input(input_shape)

    # Encoder (Downsampling Path) - Stays the same
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)

    # Decoder (Upsampling Path) - Stays the same
    up5 = concatenate([UpSampling2D(size=(2, 2))(drop4), conv3], axis=-1)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)

    # Output layer (CRITICAL CHANGE: 2 channels for Boundary + Distance Map)
    outputs = Conv2D(OUTPUT_CHANNELS, (1, 1), activation='sigmoid')(conv7)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss=multi_head_loss,
                  metrics=['accuracy'])

    return model


In [ ]:

# ==============================================================================
# 4. EXECUTION
# ==============================================================================

if __name__ == '__main__':

    # 1. Mount Google Drive and check for dependencies (you must run these in Colab)
    # from google.colab import drive
    # drive.mount('/content/drive')
    # !pip install rasterio scipy scikit-image # All needed for the new pipeline

    # 2. Data Loading and Preprocessing
    print("\n--- Starting Data Loading and Preprocessing (Step 1) ---")

    # Load and stack all 5 Sentinel-2 GeoTIFFs into one massive (H, W, 30) input array
    full_img_stack = load_all_input_tiffs(INPUT_IMAGE_NAMES, DRIVE_PATH)

    if full_img_stack is None:
        print("Aborting training due to missing Sentinel-2 input files.")
        exit()

    # Get the target shape (H, W) from the loaded input stack
    target_shape_hw = full_img_stack.shape[:2]
    print(f"Target mask shape set to: {target_shape_hw}")

    # --- CRITICAL NEW STEP FOR INSTANCE SEGMENTATION LABEL GENERATION ---
    print("\n--- Generating 2-Channel Instance Label Map from FILLED MASKS ---")

    # This function handles the resizing, boundary finding, distance map calculation, and stacking.
    full_mask_2ch = generate_instance_labels(MASK_FILE_NAMES, DRIVE_PATH, target_shape_hw)

    if full_mask_2ch is None:
        print("Aborting training due to missing or uncombinable mask files.")
        exit()

    print(f"2-channel (Boundary + Distance) label map created with shape: {full_mask_2ch.shape}")
    # ----------------------------------------------------


    # 3. Tiling and Splitting
    print("\n--- Tiling Data (Step 2) ---")
    X_tiles, Y_tiles = create_tiles(full_img_stack, full_mask_2ch, TILE_SIZE)

    # Split data into training and validation sets
    X_train, X_val, Y_train, Y_val = train_test_split(
        X_tiles, Y_tiles, test_size=VALIDATION_SPLIT, random_state=42
    )
    print(f"Training set size: {X_train.shape[0]} tiles")
    print(f"Validation set size: {X_val.shape[0]} tiles")


    # 4. Model Setup and Training
    print("\n--- Training U-Net Model (Step 3) ---")
    input_shape = (TILE_SIZE, TILE_SIZE, NUM_CHANNELS)
    unet_model = build_unet(input_shape)

    history = unet_model.fit(
        X_train,
        Y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=1,
        validation_data=(X_val, Y_val),
        shuffle=True
    )

    print("\n✅ Training complete.")

    # 5. Save the trained model
    model_save_path = os.path.join(DRIVE_PATH, 'crop_boundary_instance_unet_model.keras')
    unet_model.save(model_save_path)
    print(f"Model saved to: {model_save_path}")

In [ ]:

# ==============================================================================
# 3. U-NET ARCHITECTURE (MULTI-HEAD)
# ==============================================================================

# Custom Loss Function to handle the two different output types
def multi_head_loss(y_true, y_pred):
    """
    Loss function for the 2-channel output:
    Channel 0 (Boundary Mask): Binary Crossentropy
    Channel 1 (Distance Map): Mean Squared Error
    """
    # Channel 0: Boundary Mask (Binary Segmentation)
    y_true_boundary = y_true[:, :, :, 0]
    y_pred_boundary = y_pred[:, :, :, 0]

    # We use a weight map to focus the loss on the boundary pixels (y_true_boundary)
    # Binary Crossentropy for the boundary prediction
    boundary_loss = K.binary_crossentropy(y_true_boundary, y_pred_boundary)

    # Channel 1: Distance Map (Regression)
    y_true_distance = y_true[:, :, :, 1]
    y_pred_distance = y_pred[:, :, :, 1]

    # Mean Squared Error for the distance prediction
    distance_loss = K.mean(K.square(y_pred_distance - y_true_distance), axis=-1)

    # Combine the losses (weights can be adjusted if one task dominates)
    total_loss = boundary_loss + (0.5 * distance_loss)
    return total_loss

def build_unet(input_shape):
    """Defines a Multi-Head U-Net architecture for instance segmentation hint prediction."""
    inputs = Input(input_shape)

    # Encoder (Downsampling Path) - Stays the same
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)

    # Decoder (Upsampling Path) - Stays the same
    up5 = concatenate([UpSampling2D(size=(2, 2))(drop4), conv3], axis=-1)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)

    # Output layer (CRITICAL CHANGE: 2 channels for Boundary + Distance Map)
    outputs = Conv2D(OUTPUT_CHANNELS, (1, 1), activation='sigmoid')(conv7)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss=multi_head_loss,
                  metrics=['accuracy'])

    return model


In [ ]:
import scipy.ndimage as ndi # You must add this import!

def generate_distance_map(binary_mask_1ch):
    """
    Generates a 2-channel label mask (Boundary + Distance).
    Channel 0: Original binary mask (Boundary).
    Channel 1: Euclidean Distance Transform (for instance separation).
    """
    # 1. Compute the Euclidean Distance Transform on the NEGATIVE of the mask.
    # This measures the distance from every background pixel to the nearest field boundary.
    # The `binary_mask_1ch` must be inverted (0 for foreground, 1 for background)
    # for the distance to measure the distance to the field interior.

    # We actually want the distance from the boundary *into* the field area.
    # Assuming your original GeoTIFF masks were 1 inside the polygon and 0 outside.
    # We want distance from 0 (boundary/background) to the interior (1).

    # We need a mask where the boundary is 0 and the interior is 1.

    # A simple boundary mask (Y_train[:,:,0]) is 1 only on the boundary.
    # To get the distance to the interior, we compute the distance transform on the interior pixels.
    # For instance segmentation, it's common to compute the distance from the background
    # to the instance boundary, then invert and normalize it.

    # Simplified approach for demonstration (requires the actual field interiors):
    # This requires running a second Earth Engine export to get the *filled* polygon area,
    # not just the boundary line you currently have.
    # Given we only have the *boundary* right now, let's stick to the simplest
    # instance separation hint: the distance from the predicted boundary.

    # For now, let's assume the interior mask (1=inside, 0=outside) is available.
    # Since you only provided boundary masks, this part is tricky.

    # ----------------------------------------------------
    # TEMPORARY INSTANCE HINT (DISTANCE FROM BOUNDARY)
    # ----------------------------------------------------

    # 1. Use the binary boundary mask (1 where there is a line, 0 otherwise)
    boundary_binary = binary_mask_1ch[:,:,0]

    # 2. Compute distance from the boundary (where boundary_binary == 1)
    # The distance transform measures the distance from 0-valued pixels.
    # We invert the boundary mask: 0 (boundary) -> 1 (background), 1 (boundary) -> 0
    # The current mask has 1 for boundary, 0 for background.
    # We want distance from the nearest boundary pixel (which is 1).
    # Use the distance transform on the inverted mask to get distance *to* the boundary:
    distance_to_boundary = ndi.distance_transform_edt(1 - boundary_binary)

    # 3. Normalize the distance map for better network learning (0 to 1)
    distance_max = distance_to_boundary.max()
    distance_map = distance_to_boundary / distance_max if distance_max > 0 else distance_to_boundary

    # 4. Stack them back up: Channel 0 is boundary, Channel 1 is normalized distance
    label_2ch = np.dstack([boundary_binary, distance_map])

    return label_2ch


# --- In __main__ (REPLACING THE PLACEHOLDER) ---
# You need to run: !pip install scipy first

# Load the binary boundary mask
full_boundary_mask_1ch = load_and_combine_masks(MASK_FILE_NAMES, DRIVE_PATH)
# Generate the 2-channel label (Boundary + Distance)
full_mask_2ch = generate_distance_map(full_boundary_mask_1ch)


In [ ]:

# ==============================================================================
# 4. EXECUTION
# ==============================================================================

if __name__ == '__main__':

    # 1. Mount Google Drive and check for dependencies (you must run these in Colab)
    # from google.colab import drive
    # drive.mount('/content/drive')
    # !pip install rasterio scipy # You MUST install scipy now!

    # 2. Data Loading and Preprocessing
    print("\n--- Starting Data Loading and Preprocessing (Step 1) ---")

    # Load and stack all 5 Sentinel-2 GeoTIFFs into one massive (H, W, 30) input array
    full_img_stack = load_all_input_tiffs(INPUT_IMAGE_NAMES, DRIVE_PATH)

    # Load and combine all polygon GeoTIFFs into one massive (H, W, 1) mask array (Boundary only)
    full_boundary_mask_1ch = load_and_combine_masks(MASK_FILE_NAMES, DRIVE_PATH)

    if full_img_stack is None or full_boundary_mask_1ch is None:
        print("Aborting training due to missing input files.")
        exit()

    # --- CRITICAL NEW STEP FOR INSTANCE SEGMENTATION ---
    print("\n--- Generating 2-Channel Instance Label Map (Boundary + Distance) ---")

    # We need a function that takes full_boundary_mask_1ch and adds the distance map.
    # THIS CODE IS A PLACEHOLDER. You need to implement the distance transform here.
    # Since the necessary library `scipy.ndimage` is not imported, we skip the actual
    # creation here and explain the next step.

    # ⚠️ Placeholder: This needs to be replaced with the actual 2-channel mask generation
    # where Y_train[:, :, 0] = boundary, and Y_train[:, :, 1] = distance map.
    full_mask_2ch = np.dstack([full_boundary_mask_1ch, full_boundary_mask_1ch * 0.5])
    print(f"Placeholder 2-channel mask created with shape: {full_mask_2ch.shape}")
    # ----------------------------------------------------


    # 3. Tiling and Splitting
    print("\n--- Tiling Data (Step 2) ---")
    X_tiles, Y_tiles = create_tiles(full_img_stack, full_mask_2ch, TILE_SIZE)

    # Split data into training and validation sets
    X_train, X_val, Y_train, Y_val = train_test_split(
        X_tiles, Y_tiles, test_size=VALIDATION_SPLIT, random_state=42
    )
    print(f"Training set size: {X_train.shape[0]} tiles")
    print(f"Validation set size: {X_val.shape[0]} tiles")


    # 4. Model Setup and Training
    print("\n--- Training U-Net Model (Step 3) ---")
    input_shape = (TILE_SIZE, TILE_SIZE, NUM_CHANNELS)
    unet_model = build_unet(input_shape)

    history = unet_model.fit(
        X_train,
        Y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=1,
        validation_data=(X_val, Y_val),
        shuffle=True
    )

    print("\n✅ Training complete.")

    # 5. Save the trained model
    model_save_path = os.path.join(DRIVE_PATH, 'crop_boundary_instance_unet_model.keras')
    unet_model.save(model_save_path)
    print(f"Model saved to: {model_save_path}")